In [1]:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet



target_shape = (224, 224)

In [2]:
# img_width = 224
# img_height = 224
# tf.random.set_seed(42)
# val_size = 0.2
# def preprocess_image(filename):
#     image_string = tf.io.read_file(filename)
#     image = tf.image.decode_jpeg(image_string, channels=3)
#     image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_flip_up_down(image)
#     #image = tf.image.random_jpeg_quality(image, 75, 95)
#     image = tf.image.resize(image, (img_width, img_height))
#     #image = image.numpy()
#     #image = np.rint(image)
#     return image

# def preprocess_triplets(anchor, positive, negative):
#     return (
#         preprocess_image(anchor),
#         preprocess_image(positive),
#         preprocess_image(negative),
#     )
def preprocess_image(filename):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """

    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    return image


def preprocess_triplets(anchor, positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(anchor),
        preprocess_image(positive),
        preprocess_image(negative),
    )

In [6]:
test_triplets = pd.read_csv(r'C:\Users\Manue\Documents\GitHub\IML-Projects\Project 3\task3_be9ai3nsdj\test_triplets.txt', sep=' ', header=None)
train_triplets = pd.read_csv(r'C:\Users\Manue\Documents\GitHub\IML-Projects\Project 3\task3_be9ai3nsdj\train_triplets.txt', sep=' ', header=None)
#number_of_images=len(train_triplets.iloc[:,0])
anchor_indexes=train_triplets.iloc[:,0]
display(len(anchor_indexes))
display(anchor_indexes)
positives_indexes=train_triplets.iloc[:,1]
negatives_indexes=train_triplets.iloc[:,2]
test_anchor_indexes=test_triplets.iloc[:,0]
test_positives_indexes=test_triplets.iloc[:,1]
test_negatives_indexes=test_triplets.iloc[:,2]

59515

0        2461
1        2299
2        4663
3        4532
4        3454
         ... 
59510     466
59511    2646
59512    3255
59513    2136
59514    2509
Name: 0, Length: 59515, dtype: int64

# New Section

In [7]:
str_path = r'C:\Users\Manue\Documents\GitHub\IML-Projects\Project 3\task3_be9ai3nsdj\food'
anchor_images=list()
for number in anchor_indexes:
    anchor_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(anchor_images[0])
display(anchor_images[0:3])
anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_images)

positives_images=list()
for number in positives_indexes:
    positives_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(positives_images[0])
display(positives_images[0:3])
positive_dataset = tf.data.Dataset.from_tensor_slices(positives_images)

negatives_images=list()
for number in negatives_indexes:
    negatives_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(negatives_images[0])
display(negatives_images[0:3])
negative_dataset = tf.data.Dataset.from_tensor_slices(negatives_images)

test_anchor_images=list()
for number in test_anchor_indexes:
    test_anchor_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(test_anchor_images[0])
display(test_anchor_images[0:3])
test_anchor_dataset = tf.data.Dataset.from_tensor_slices(test_anchor_images)

test_positives_images=list()
for number in test_positives_indexes:
    test_positives_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(test_positives_images[0])
display(test_positives_images[0:3])
test_positive_dataset = tf.data.Dataset.from_tensor_slices(test_positives_images)

test_negatives_images=list()
for number in test_negatives_indexes:
    test_negatives_images.append(str_path+'/'+str(number).rjust(5, '0')+'.jpg')
image_string = tf.io.read_file(test_negatives_images[0])
display(test_negatives_images[0:3])
test_negative_dataset = tf.data.Dataset.from_tensor_slices(test_negatives_images)

image_count = len(anchor_images)
test_image_count = len(test_anchor_images)
display(len(anchor_images))
display(len(test_anchor_images))

['C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/02461.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/02299.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/04663.jpg']

['C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/03450.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/02499.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/01056.jpg']

['C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/02678.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/04987.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/03029.jpg']

['C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/09896.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/06592.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/08655.jpg']

['C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/09640.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/09283.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/06174.jpg']

['C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/09177.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/07104.jpg',
 'C:\\Users\\Manue\\Documents\\GitHub\\IML-Projects\\Project 3\\task3_be9ai3nsdj\\food/06400.jpg']

59515

59544

In [8]:
dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))
dataset = dataset.shuffle(buffer_size=60000)
dataset = dataset.map(preprocess_triplets)

# Let's now split our dataset in train and validation.
train_dataset = dataset.take(round(image_count * 0.8))
val_dataset = dataset.skip(round(image_count * 0.8))

train_dataset = train_dataset.batch(1024, drop_remainder=False)
train_dataset = train_dataset.prefetch(256)

val_dataset = val_dataset.batch(8192, drop_remainder=False)
val_dataset = val_dataset.prefetch(2048)

In [9]:
dataset = tf.data.Dataset.zip((test_anchor_dataset, test_positive_dataset, test_negative_dataset))
dataset = dataset.shuffle(buffer_size=60000)
dataset = dataset.map(preprocess_triplets)

# Let's now split our dataset in train and validation.
test_dataset = dataset.take(test_image_count)


test_dataset = test_dataset.batch(1024, drop_remainder=False)
test_dataset = test_dataset.prefetch(256)

In [10]:
base_cnn = resnet.ResNet50(
    weights="imagenet", input_shape=target_shape + (3,), include_top=False
)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
#dense1 = layers.Dropout(0.2)(dense1)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
#dense2 = layers.Dropout(0.2)(dense2)
dense2 = layers.BatchNormalization()(dense2)
# dense3 = layers.Dense(128, activation="relu")(dense2)
# dense3 = layers.BatchNormalization()(dense3)
output = layers.Dense(128)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")

trainable = False
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

In [11]:
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
positive_input = layers.Input(name="positive", shape=target_shape + (3,))
negative_input = layers.Input(name="negative", shape=target_shape + (3,))

distances = DistanceLayer()(
    embedding(resnet.preprocess_input(anchor_input)),
    embedding(resnet.preprocess_input(positive_input)),
    embedding(resnet.preprocess_input(negative_input)),
)

siamese_network = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

In [12]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

In [13]:
siamese_model = SiameseModel(siamese_network)
siamese_model.compile(optimizer=optimizers.Adam(0.0001))
siamese_model.fit(train_dataset, epochs=1)

In [2]:
acc_list=list()
for i in range(1):
    sample = next(iter(val_dataset))
    #visualize(*sample)

    anchor, positive, negative = sample
    anchor_embedding, positive_embedding, negative_embedding = (
        embedding(resnet.preprocess_input(anchor)),
        embedding(resnet.preprocess_input(positive)),
        embedding(resnet.preprocess_input(negative)),
    )
    ap_distance = tf.reduce_sum(tf.square(anchor_embedding - positive_embedding), -1)
    an_distance = tf.reduce_sum(tf.square(anchor_embedding - negative_embedding), -1)
    counter=0
    #calculate accuracies for 10 batches
    for i in range(len(ap_distance)):
        if ap_distance[i]<an_distance[i]:
            counter=counter+1
    display(counter/len(ap_distance))
    acc_list.append(counter/len(ap_distance))
display("acc= ", np.mean(acc_list))

NameError: ignored

In [3]:
pred_list=list()
for i in range(10):
    sample = next(iter(val_dataset))
    #visualize(*sample)

    anchor, positive, negative = sample
    anchor_embedding, positive_embedding, negative_embedding = (
        embedding(resnet.preprocess_input(anchor)),
        embedding(resnet.preprocess_input(positive)),
        embedding(resnet.preprocess_input(negative)),
    )
    ap_distance = tf.reduce_sum(tf.square(anchor_embedding - positive_embedding), -1)
    an_distance = tf.reduce_sum(tf.square(anchor_embedding - negative_embedding), -1)
    counter=0
    #calculate accuracies for 10 batches
    for i in range(len(ap_distance)):
          pred_list.append(int(ap_distance[i].numpy()<an_distance[i].numpy()))
display(pred_list)
display(np.mean(pred_list))

NameError: ignored

In [4]:
display(np.mean(pred_list))

NameError: ignored